In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
block_size = 8
batch_size = 4
max_iters = 1000
learning_rate = 3e-4
eval_interval = 250
n_embd = 384        
n_layer=4
dropout=0.2

In [3]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    chars = f.read()
vocab_size = len(set(chars))

In [4]:
string_to_int= {ch:i for i,ch in enumerate(chars)}
int_to_string= {i:ch for i,ch in enumerate(chars)}
encode=lambda s: [string_to_int[c] for c in s]
decode=lambda l: ''.join([int_to_string[i] for i in l])
data=torch.tensor(encode(chars), dtype=torch.long, device=device)

In [6]:
n = int(len(data) * 0.8)
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i + block_size] for i in ix])
    y = torch.stack([data[i + 1:i + block_size + 1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y


In [5]:
@torch.no_grad()
def estimate_loss(split):
    out = {}
    model=eval()
    for split in  ['train','val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            x, y = get_batch(split)
            logits, loss = model(x, y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [ ]:
class FeedForward(nn.Module):
    """ A simple feed forward layer followed by a non linearity """
    def __init__(self,n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4*n_embd), #4x the embedding size
            nn.ReLU(),
            nn.Linear(4*n_embd, n_embd)
            nn.Dropout(dropout) #dropout layer to prevent overfitting
        )

    def forward(self, x):
        return self.net(x)


class Block(nn.Module):
    """ Transformer block: communication followed by computation """
    def __init__(self,n_embd,n_head):
        #n_embd: embedding dimension, n_head: number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head #size of each head
        self.sa = MultiHeadAttention(n_head, head_size) #self attention layer
        self.ffwd = FeedForward(n_embd) #feed forward layer
        self.ln1 = nn.LayerNorm(n_embd) #layer norm after self attention
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        y=self.sa(x) #self attention layer
        x = self.ln1(x+y) 
        y=self.ffwd(x) 
        x = self.ln2(x+y)
        return x

class GPTLanguageModel(nn.Module):
    def __init__(self,vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd,n_head=n_head) for _ in range(n_layer)]) #4 decoder blocks
        self.ln_f = nn.LayerNorm(n_embd) #final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size) #head to predict the next token 
        self.apply(self._init_weights) #apply the weight initialization function to all layers

    def _init_weights(self,module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight,mean=0.0, std=0.02)
            if module.bias is not None:
                module.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, index, targets=None):
        logits=self.token_embedding_table(index)

        #idx and targets are both (B,T) tensor of integers
        tok_emb=self.token_embedding_table(index) # (B,T,C)
        pos_emb=self.position_embedding_table(torch.arange(block_size, device=device)) # (T,C)
        x=tok_emb+pos_emb # (B,T,C)
        x=self.blocks(x) # (B,T,C)
        x=self.ln_f(x) # (B,T,C)
        logits=self.lm_head(x) # (B,T,vocab_size)
        if targets is None:
            return logits
        else:
            B,T,C=logits.shape
            logits=logits.view(B*T,C)
            targets=targets.view(B*T)
            loss=F.cross_entropy(logits,targets)
        return logits,loss
    def generate(self,index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits,loss = self.forward(index)
            logits = logits[:, -1, :] 
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index=torch.cat((index, index_next), dim=1)
        return index
model = GPTLanguageModel(vocab_size)
m=model.to(device)

In [ ]:
#Pytorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=learning_rate)
for iter in range(max_iters):
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"iter {iter}: train loss {losses['train']:.3f}, val loss {losses['val']:.3f}")

    xb, yb = get_batch('train')
    logits, loss = model.forward(xb, yb)

    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())